In [1]:
import pandas as pd
import numpy as np
# from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
num_random_seeds = 5
num_bootstrap_models = 100

In [8]:
data_source = "data/obermeyer/obermeyer_data_cleaned.csv"
prediction_output = "predictions/obermeyer/model_predictions_25.csv"
target_variable = "cost_t"
features = ['dem_female', 'dem_age_band_18-24_tm1', 'dem_age_band_25-34_tm1', 'dem_age_band_35-44_tm1', 'dem_age_band_45-54_tm1',
            'dem_age_band_55-64_tm1', 'dem_age_band_65-74_tm1', 'dem_age_band_75+_tm1', 'hypertension_elixhauser_tm1', 'cost_dialysis_tm1',
            'cost_emergency_tm1', 'cost_home_health_tm1', 'cost_ip_medical_tm1', 'cost_ip_surgical_tm1', 'cost_laboratory_tm1',
            'cost_op_primary_care_tm1', 'cost_op_specialists_tm1', 'cost_op_surgery_tm1', 'cost_other_tm1', 'cost_pharmacy_tm1',
            'cost_physical_therapy_tm1', 'cost_radiology_tm1', 'gagne_sum_tm1', 'person_id', 'threshold_25', 'threshold_50', 'threshold_75']

In [9]:
df = pd.read_csv(data_source)
X = df[features]
y = df[target_variable]

In [10]:
df['person_id'].isna().sum()

0

In [11]:
def get_bootstrap_model(X_train, y_train, X_test, random_seed, bootstrap_size=0.5):
    np.random.seed(random_seed)
    idx = np.random.permutation(len(X_train))[:int(len(X_train)*bootstrap_size)]
    model = LinearRegression()
    # model = RandomForest(n_estimators=25, random_state=random_seed)
    model.fit(X_train[idx], y_train[idx])
    scores = model.predict(X_test)
    # scores = model.predict_proba(X_test)[:, 1]
    return scores

In [12]:
output = []
for random_seed in range(num_random_seeds):
    print("random seed", random_seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=random_seed)
    threshold_25 = X_test['threshold_25'].to_numpy()
    threshold_50 = X_test['threshold_50'].to_numpy()
    threshold_75 = X_test['threshold_75'].to_numpy()
    test_idx = X_test['person_id'].to_numpy()
    # print(test_idx.isna().sum())
    X_train = X_train.drop(columns=['person_id']).to_numpy()
    y_train = y_train.to_numpy()
    X_test = X_test.drop(columns=['person_id']).to_numpy()
    y_test = y_test.to_numpy()

    predictions = []
    columns = []
    accuracy = []
    for i in tqdm(range(num_bootstrap_models)):
        columns.append(f'm_{i+1}')
        scores = get_bootstrap_model(X_train, y_train, X_test, random_seed=i, bootstrap_size=0.5)
        predictions.append(scores)
        accuracy.append(float((np.round(scores)==y_test).sum() / len(y_test)))
    
    predictions_df = pd.DataFrame(predictions).transpose()
    predictions_df.columns=columns
    print("predictions length", len(predictions_df))
    predictions_df["y"] = y_test
    print("index length", len(test_idx))
    predictions_df["idx"] = test_idx 
    predictions_df["seed"] = random_seed
    predictions_df['threshold_25'] = threshold_25
    predictions_df['threshold_50'] = threshold_50
    predictions_df['threshold_75'] = threshold_75
    output.append(predictions_df)
output = pd.concat(output)

random seed 0


100%|██████████| 100/100 [00:19<00:00,  5.13it/s]


predictions length 16099
index length 16099
random seed 1


100%|██████████| 100/100 [00:19<00:00,  5.22it/s]


predictions length 16099
index length 16099
random seed 2


...

predictions length 16099
index length 16099
random seed 3


100%|██████████| 100/100 [00:15<00:00,  6.54it/s]


predictions length 16099
index length 16099
random seed 4


100%|██████████| 100/100 [00:20<00:00,  4.78it/s]


predictions length 16099
index length 16099


In [13]:
output.to_csv(prediction_output, index=False)

In [23]:
output.head()

,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,...,m_94,m_95,m_96,m_97,m_98,m_99,m_100,y,idx,seed
0,0.006753,0.008166,0.007790,0.008346,0.007259,0.007138,0.008137,0.008380,0.006640,0.007848,...,0.008560,0.007984,0.007430,0.008216,0.007169,0.007849,0.007113,0.009628,2545,0
1,0.013049,0.013692,0.013846,0.014468,0.014478,0.014450,0.014055,0.013888,0.014334,0.014554,...,0.014325,0.014512,0.013678,0.013755,0.013935,0.014385,0.014230,0.004905,8198,0
2,0.021138,0.018986,0.018488,0.018745,0.017153,0.019729,0.018739,0.017566,0.018916,0.019219,...,0.019216,0.020513,0.019854,0.018457,0.018957,0.019271,0.019365,0.009446,46461,0
3,0.002706,0.002252,0.002763,0.002001,0.003470,0.001979,0.002444,0.002950,0.002548,0.002147,...,0.003356,0.003260,0.002520,0.002663,0.003136,0.002370,0.002492,0.002361,30620,0
4,0.009980,0.012810,0.014430,0.010169,0.011937,0.010077,0.014131,0.012601,0.011287,0.011969,...,0.011391,0.010872,0.013481,0.012869,0.011242,0.012343,0.011475,0.003996,47418,0
